In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the ticker
tickerSymbol = 'DOCS'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
data = tickerData.history(period='1d', start='2010-1-1', end='2023-1-25')

# We will predict 'Close' price, hence 'Close' will be our label
labels = data['Close']

# Drop 'Close' column from data as it will be predicted
data = data.drop(['Close'], axis=1)

# Prepare data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
scaled_labels = scaler.fit_transform(labels.values.reshape(-1,1))

# Define look back period
look_back = 60

# Prepare training data
x = []
y = []

for i in range(look_back, len(scaled_data)):
    x.append(scaled_data[i-look_back:i])
    y.append(scaled_labels[i])

x, y = np.array(x), np.array(y)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Make predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)  # unscaling the predictions

# Initialize counters
buy_count = 0
sell_count = 0
hold_count = 0

# Analyze predictions
for i in range(len(predictions)):
    # If prediction is higher than actual price by a certain threshold, we consider it a buy
    if predictions[i][0] > y_test[i] * 1.03:  # 3% increase threshold for example
        buy_count += 1
    # If prediction is lower than actual price by a certain threshold, we consider it a sell
    elif predictions[i][0] < y_test[i] * 0.97:  # 3% decrease threshold for example
        sell_count += 1
    # If prediction is within the actual price range, we consider it a hold
    else:
        hold_count += 1

# Compute the aggregate verdict
verdict_counts = {"BUY": buy_count, "SELL": sell_count, "HOLD": hold_count}
aggregate_verdict = max(verdict_counts, key=verdict_counts.get)

print(f"Aggregate verdict: {aggregate_verdict}")


2023-06-01 20:43:30.284501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-01 20:43:30.287176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-01 20:43:30.288885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

271/271 [==============================] - 12s 35ms/step - loss: 0.0078


2023-06-01 20:43:42.458000: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-01 20:43:42.460078: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-01 20:43:42.463809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - 1s 37ms/step
Aggregate verdict: BUY


In [ ]:
# import libraries
import yfinance as yf
import pandas as pd
from fbprophet import Prophet
from NewsSentiment import TargetSentimentClassifier
import fundamentalanalysis as fa

# define the ticker symbol
tickerSymbol = 'MSFT'

# get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2010-1-1', end='2020-1-25')

# reset the index, we want to use dates in the 'ds' column
tickerDf.reset_index(level=0, inplace=True)

# Prepare for Prophet model
tickerDf = tickerDf.rename(columns={'Date':'ds', 'Close':'y'})

# Perform sentiment analysis
tsc = TargetSentimentClassifier()
# Replace this with real news data. For now, this is just a placeholder
news = "Microsoft announced record earnings for Q2 2020"
sentiment = tsc.infer_from_text(news, tickerSymbol)
print(sentiment[0])

# Get financial ratios
api_key = "823653952fa1e2ec38c0e9c575b3a798"  # Replace with your API Key
profile = fa.profile(tickerSymbol, api_key)
quotes = fa.quote(tickerSymbol, api_key)
financial_ratios_annually = fa.financial_ratios(tickerSymbol, api_key, period="annual")
pe_ratio = financial_ratios_annually['priceEarningsRatio']

# You can add sentiment and PE ratio into your model here. For example, you can add them as additional regressors in your Prophet model
# Or you can use them in a separate model to predict whether to buy, sell, or hold.

# Create and train Prophet model
m = Prophet(daily_seasonality=True)
m.fit(tickerDf)

# Specify the number of days in future
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

# Output the prediction
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
